## MELANOMA DEEP LEARNING IMAGE CLASSIFICATION
### w251 - Fall 2021

![iphone capture](iphone_capture.PNG)

In [1]:
import os
import time
import random
import argparse
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import sklearn
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from util import GradualWarmupSchedulerV2
#import apex
#from apex import amp
from diag_dataset import get_df_diag, get_transforms, MelanomaDataset
from models import Effnet_Melanoma, Resnest_Melanoma, Seresnext_Melanoma
from train import get_trans
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageOps

In [2]:
#KERNEL_TYPE = '9c_b3_768_512_ext_18ep'
KERNEL_TYPE = 'xxx'
DATA_DIR ='./diagnosis'
DATA_FOLDER = 768
IMAGE_SIZE = 256
ENET_TYPE = 'efficientnet_b3'
BATCH_SIZE = 64
NUM_WORKERS = 1
OUT_DIM = 9
USE_AMP = True
DEBUG = True
MODEL_DIR = './weights'
LOG_DIR = './logs'
SUB_DIR = './subs'
# choices=['best', 'best_20', 'final'], default="best")
EVAL = 'best' 
N_TEST = 8
# We only have one GPU on the ol' Jetson
CUDA_VISIBLE_DEVICES = '0'

In [3]:
os.makedirs(SUB_DIR, exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = CUDA_VISIBLE_DEVICES

if ENET_TYPE == 'resnest101':
    ModelClass = Resnest_Melanoma
elif ENET_TYPE == 'seresnext101':
    ModelClass = Seresnext_Melanoma
elif 'efficientnet' in ENET_TYPE:
    ModelClass = Effnet_Melanoma
else:
    raise NotImplementedError()

DP = len(os.environ['CUDA_VISIBLE_DEVICES']) > 1

device = torch.device('cuda')

In [13]:
def main():

    df_diag, mel_idx = get_df_diag(
        KERNEL_TYPE,
        DATA_DIR
    )
    print('printing df_diag')
    print(df_diag)
    print('printing mel_idx')
    print(mel_idx)

    transforms_train, transforms_val = get_transforms(IMAGE_SIZE)

    #if DEBUG:
    #    df_test = df_test.sample(BATCH_SIZE * 3)
    dataset_diag = MelanomaDataset(df_diag, 'test', transform=transforms_val)
    diag_loader = torch.utils.data.DataLoader(dataset_diag, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

    # load model
    models = []
    for fold in range(5):

        if EVAL == 'best':
            model_file =  os.path.join(MODEL_DIR, 'baseline-model-1_final_fold4.pth') 
        elif EVAL == 'best_20': 
            model_file = os.path.join(MODEL_DIR, 'best_20_model.pth')
        if EVAL == 'final':
            model_file = os.path.join(MODEL_DIR, 'final_model.pth')

        model = ModelClass(
            ENET_TYPE,
            out_dim=OUT_DIM
        )
        model = model.to(device)

        try:  # single GPU model_file
            model.load_state_dict(torch.load(model_file), strict=True)
        except:  # multi GPU model_file
            state_dict = torch.load(model_file)
            state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
            model.load_state_dict(state_dict, strict=True)
        
        if len(os.environ['CUDA_VISIBLE_DEVICES']) > 1:
            model = torch.nn.DataParallel(model)

        model.eval()
        models.append(model)

    # predict
    PROBS = []
    with torch.no_grad():
        for (data) in tqdm(diag_loader):
            data = data.to(device)
            probs = torch.zeros((data.shape[0], OUT_DIM)).to(device)
            for model in models:
                for I in range(N_TEST):
                    l = model(get_trans(data, I))
                    probs += l.softmax(1)

            probs /= N_TEST
            probs /= len(models)

            PROBS.append(probs.detach().cpu())

    PROBS = torch.cat(PROBS).numpy()
    
    print('printing PROBS')
    print(PROBS)
    
    # save cvs
    df_diag['target'] = PROBS[:, mel_idx]
    df_diag[['image_name', 'target']].to_csv(os.path.join(SUB_DIR, 'results.csv'), index=False)
    
    images = df_diag['filepath']
    
    print("")
    
    print("OUTPUT RESULTS")
    
    fig = plt.figure(figsize=(20,5))
    for i in range(len(images)):
        ax = fig.add_subplot(1, len(images), i+1)
        ax.axis('off')
        img = Image.open(images[i])
        if df_diag['target'][i] < .005:
            border_img = ImageOps.expand(img, border=25, fill='lightgreen')
        else:
            border_img = ImageOps.expand(img, border=25, fill='indianred')
        imgplot = plt.imshow(border_img)
        ax.set_title(str(round(float(df_diag['target'][i]),4)))

## CAPTURING IMAGES FOR DIAGNOSIS


INPUT IMAGES FOR DIAGOSIS


In [14]:
df_diag, mel_idx = get_df_diag(
    KERNEL_TYPE,
    DATA_DIR
)
#print('printing df_test')
#print(df_test)

images = df_diag['filepath']

print("INPUT IMAGES FOR DIAGOSIS")
fig = plt.figure(figsize=(20,5))
for i in range(len(images)):
    ax = fig.add_subplot(1, len(images), i+1)
    ax.axis('off')
    img = Image.open(images[i])
    if df_diag['target'][i] < 1:
        border_img = ImageOps.expand(img, border=25, fill='lightgreen')
    else:
        border_img = ImageOps.expand(img, border=25, fill='indianred')
    imgplot = plt.imshow(border_img)
    ax.set_title(df_diag['target'][i])

## SCANNING IMAGES FOR MELANOMA

printing df_diag
     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male          45                     head/neck   
1  ISIC_0149568  IP_0962375  female          55               upper extremity   
2  ISIC_0156772  IP_3586223  female          30                         torso   
3  ISIC_0188432  IP_0135517  female          50               upper extremity   

  diagnosis benign_malignant  target  tfrecord  width  height  \
0   unknown           benign       8         0   6000    4000   
1  melanoma        malignant       8         0   6000    4000   
2     nevus           benign       5         0   3264    2448   
3  melanoma        malignant       8         5   3264    2448   

                            filepath  
0  ./diagnosis/diag/ISIC_2637011.jpg  
1  ./diagnosis/diag/ISIC_0149568.jpg  
2  ./diagnosis/diag/ISIC_0156772.jpg  
3  ./diagnosis/diag/ISIC_0188432.jpg  
printing mel_idx
4


100%|██████████| 1/1 [00:12<00:00, 12.61s/it]


printing PROBS
[[1.90604135e-06 1.11844918e-06 3.32145028e-05 8.85104528e-05
  1.42151795e-04 3.79453617e-04 8.99335835e-04 5.34657102e-06
  9.98448849e-01]
 [9.77646705e-05 6.27442132e-05 2.72918283e-03 1.40735836e-04
  5.11897786e-04 7.85285886e-03 1.04691265e-02 1.35265652e-03
  9.76783097e-01]
 [1.72318978e-04 1.33469046e-04 4.92439000e-03 7.61842413e-04
  2.42631864e-02 4.61310118e-01 1.45766763e-02 1.47724233e-04
  4.93710369e-01]
 [2.01488758e-04 4.67745340e-05 1.72752276e-01 4.85292461e-04
  1.49367347e-01 7.42909079e-03 5.55464393e-03 1.98649149e-03
  6.62176669e-01]]

OUTPUT RESULTS


In [15]:
main()